In [ ]:
pip install experta

In [ ]:
import collections.abc
if not hasattr(collections, 'Mapping'):
    collections.Mapping = collections.abc.Mapping

In [ ]:
from experta import *

# === Definición de hechos base ===
class Hombre(Fact):
    """Representa a un hombre en la familia"""
    pass

class Mujer(Fact):
    """Representa a una mujer en la familia"""
    pass

class Padre(Fact):
    """Relación de paternidad"""
    pass

class Madre(Fact):
    """Relación de maternidad"""
    pass

# === Hechos derivados por inferencia ===
class Progenitor(Fact):
    """Padre o madre"""
    pass

class Abuelo(Fact):
    """Abuelo de alguien"""
    pass

class Abuela(Fact):
    """Abuela de alguien"""
    pass

class Hermano(Fact):
    """Relación entre hermanos hombres"""
    pass

class Hermana(Fact):
    """Relación entre hermanas mujeres"""
    pass

class Tio(Fact):
    """Relación de tío con sobrino"""
    pass

class Tia(Fact):
    """Relación de tía con sobrino"""
    pass

class Primo(Fact):
    """Relación entre primos"""
    pass

# === Motor de inferencia basado en reglas ===
class ArbolGenealogico(KnowledgeEngine):

    def __init__(self):
        super().__init__()
        # Conjunto para llevar control de relaciones procesadas
        self.relaciones_procesadas = set()

    # === REGLAS DE INFERENCIA COMPLETAS (EJEMPLOS) ===

    @Rule(Padre(padre=MATCH.padre, hijo=MATCH.hijo), salience=100)
    def padre_a_progenitor(self, padre, hijo):
        self.declare(Progenitor(progenitor=padre, hijo=hijo))
        print(f"Progenitor: {padre} es progenitor de {hijo}")

    @Rule(Madre(madre=MATCH.madre, hijo=MATCH.hijo), salience=100)
    def madre_a_progenitor(self, madre, hijo):
        self.declare(Progenitor(progenitor=madre, hijo=hijo))
        print(f"Progenitor: {madre} es progenitor de {hijo}")

    # === REGLAS CON ESPACIOS PARA COMPLETAR ===

    @Rule(Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.hijo),
          Progenitor(progenitor=MATCH.hijo, hijo=MATCH.nieto),
          Hombre(nombre=MATCH.progenitor), salience=80)
    def inferir_abuelo(self, progenitor, nieto):
        self.declare(Abuelo(abuelo=progenitor, nieto=**))
        print(f"Abuelo: {progenitor} es abuelo de {**}")

    @Rule(Progenitor(progenitor=MATCH.** , hijo=MATCH.hijo),
          Progenitor(progenitor=MATCH.hijo, hijo=MATCH.nieto),
          Mujer(nombre=MATCH.progenitor), salience=80)
    def inferir_abuela(self, progenitor, nieto):
        self.declare(**(abuela=progenitor, nieto=nieto))
        print(f"Abuela: {progenitor} es abuela de {nieto}")

    @Rule(Progenitor(progenitor=MATCH.**, hijo=MATCH.hijo1),
          Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.hijo2),
          Hombre(nombre=MATCH.hijo1), salience=60)
    def inferir_hermano(self, progenitor, hijo1, hijo2):
        if hijo1 == **:
            return
        relacion_id = f"hermano_{hijo1}_{hijo2}"
        if relacion_id in self.**:
            return
        self.relaciones_procesadas.**(relacion_id)
        self.declare(Hermano(hermano=**, hermano_de=**))
        print(f"Hermano: {hijo1} es hermano de {hijo2}")

    @Rule(Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.hijo1),
          Progenitor(progenitor=MATCH.**, hijo=MATCH.hijo2),
          Mujer(nombre=MATCH.hijo1), salience=60)
    def inferir_hermana(self, progenitor, hijo1, hijo2):
        if hijo1 == hijo2:
            return
        relacion_id = f"hermana_{**}_{**}"
        if relacion_id in self.relaciones_procesadas:
            return
        self.relaciones_procesadas.add(***)
        self.declare(Hermana(**=hijo1, **=hijo2))
        print(f"Hermana: {hijo1} es hermana de {hijo2}")

    @Rule(Hermano(hermano=MATCH.hermano, hermano_de=MATCH.progenitor),
          Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.sobrino), salience=40)
    def inferir_tio(self, hermano, sobrino):
        self.declare(Tio(tio=**, sobrino=**))
        print(f"Tío: {hermano} es tío de {sobrino}")

    @Rule(Hermana(hermana=MATCH.hermana, hermana_de=MATCH.progenitor),
          Progenitor(progenitor=MATCH.progenitor, hijo=MATCH.sobrino), salience=40)
    def inferir_tia(self, hermana, sobrino):
        self.declare(***)
        print(f"Tía: {hermana} es tía de {sobrino}")

    @Rule(Tio(tio=MATCH.tio, sobrino=MATCH.primo1),
          Progenitor(progenitor=MATCH.tio, hijo=MATCH.primo2), salience=20)
    def inferir_primos_por_tio(self, tio, primo1, primo2):
        if primo1 == **:
            return
        if primo1 < primo2:
            relacion_id = f"primos_{**}_{**}"
        else:
            relacion_id = f"primos_{**}_{**}"
        if relacion_id in self.relaciones_procesadas:
            return
        self.relaciones_procesadas.add(relacion_id)
        self.declare(Primo(primo=primo1, primo_de=primo2))
        self.declare(Primo(primo=primo2, primo_de=primo1))
        print(f"Primos: {primo1} y {primo2} son primos")

    @Rule(Tia(tia=MATCH.tia, sobrino=MATCH.primo1),
          Progenitor(progenitor=MATCH.tia, hijo=MATCH.primo2), salience=20)
    def inferir_primos_por_tia(self, tia, primo1, primo2):
        if primo1 == primo2:
            return
        if primo1 < primo2:
            relacion_id = f"primos_{primo1}_{**}"
        else:
            relacion_id = f"primos_{primo2}_{**}"
        if relacion_id in self.**:
            return
        self.relaciones_procesadas.add(***)
        self.declare(***)
        self.declare(***)
        print(f"Primos: {primo1} y {primo2} son primos")